<a href="https://colab.research.google.com/github/mario-rot/Introduction-to-Human-Language-Technology/blob/main/Project_LaurenTucker_MarioRosas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab session 7 (Word Sequences) - ILTH

**Students:** Lauren Tucker & Mario Rosas !!!

## Loading Data

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
%%shell
git clone https://github.com/mario-rot/Introduction-to-Human-Language-Technology.git
cd 'Introduction-to-Human-Language-Technology'
mv 'Complementary Material' /content/

pip install svgling

Cloning into 'Introduction-to-Human-Language-Technology'...
remote: Enumerating objects: 864, done.
remote: Counting objects: 100% (864/864), done.
remote: Compressing objects: 100% (792/792), done.
remote: Total 864 (delta 117), reused 788 (delta 67), pack-reused 0
Receiving objects: 100% (864/864), 2.27 MiB | 14.46 MiB/s, done.
Resolving deltas: 100% (117/117), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 67 kB 3.0 MB/s 


In [2]:
import pandas as pd
from text_processing import text_processing, compute_metrics
from scipy.stats import pearsonr
import numpy as np
dt = pd.read_csv('Complementary Material/test-gold/STS.input.SMTeuroparl.txt',sep='\t',header=None)
dt['gs'] = pd.read_csv('Complementary Material/test-gold/STS.gs.SMTeuroparl.txt',sep='\t',header=None)

# Tests

#### Loading the data to the text processing class

In [19]:
sentences_1 = dt[0]
sentences_2 = dt[1]

tp1 = text_processing(sentences_1)
tp2 = text_processing(sentences_2)

In [20]:
pearsonr(dt['gs'],compute_metrics([sentences_1, sentences_2], ['jaccard']).do()[0])

PearsonRResult(statistic=-0.3971297709735512, pvalue=8.622653547988135e-19)

#### Testing with data cleaning

In [21]:
pearsonr(dt['gs'],compute_metrics([tp1.clean_data(), tp2.clean_data()], ['jaccard']).do()[0])

PearsonRResult(statistic=-0.4679440754704449, pvalue=2.358310094026997e-26)

#### Testing with tokenized data

In [22]:
pearsonr(dt['gs'],compute_metrics([tp1.tokenize_data(), tp2.tokenize_data()], ['jaccard']).do()[0])

PearsonRResult(statistic=-0.45049771693186835, pvalue=2.5356459143049236e-24)

In [23]:
'----- First group ----',tp1.frequency(True, 'cleaned'),'----- Second group -----', tp2.frequency(True, 'cleaned')

('----- First group ----',
 let            51
 report         51
 lose           51
 amendment      34
 power          34
                ..
 others         17
 separate       17
 basis          17
 accumulated    17
 hatred         17
 Length: 136, dtype: int64,
 '----- Second group -----',
 report          45
 european        41
 take            38
 parliament      37
 president       32
                 ..
 good             1
 alone            1
 offers           1
 country          1
 enthusiastic     1
 Length: 327, dtype: int64)

In [24]:
'----- First group ----',tp1.frequency(True, 'tokenized'),'----- Second group -----', tp2.frequency(True, 'tokenized')

('----- First group ----',
 .              391
 the            289
 ,              221
 of             153
 to             136
               ... 
 separate        17
 on              17
 basis           17
 accumulated     17
 hatred          17
 Length: 203, dtype: int64,
 '----- Second group -----',
 the             446
 .               390
 of              253
 to              205
 ,               144
                ... 
 country           1
 Vote              1
 h                 1
 enthusiastic      1
 about             1
 Length: 459, dtype: int64)

#### Testing with lemmas

##### With cleaned data

In [25]:
pearsonr(dt['gs'],compute_metrics([tp1.lemmatize_data('cleaned'), tp2.lemmatize_data('cleaned')], ['jaccard']).do()[0])

PearsonRResult(statistic=-0.4810231734170824, pvalue=5.904510415498371e-28)

##### With tokenized data

In [26]:
pearsonr(dt['gs'],compute_metrics([tp1.lemmatize_data('tokenized'), tp2.lemmatize_data('tokenized')], ['jaccard']).do()[0])

PearsonRResult(statistic=-0.45684718944780944, pvalue=4.766946098384746e-25)

#### Testing with most common lemma

##### With cleaned data

In [27]:
pearsonr(dt['gs'],compute_metrics([tp1.mc_lemmatize_data('cleaned'), tp2.mc_lemmatize_data('cleaned')], ['jaccard']).do()[0])

PearsonRResult(statistic=-0.4884451179547373, pvalue=6.780872227864517e-29)

##### With tokenized data

In [28]:
pearsonr(dt['gs'],compute_metrics([tp1.mc_lemmatize_data('tokenized'), tp2.mc_lemmatize_data('tokenized')], ['jaccard']).do()[0])

PearsonRResult(statistic=-0.4598193578111567, pvalue=2.1538267683062985e-25)

##### Testing with lesk

In [29]:
pearsonr(dt['gs'],compute_metrics([tp1.apply_lesk_data('cleaned'), tp2.apply_lesk_data('cleaned')], ['jaccard']).do()[0])

PearsonRResult(statistic=-0.46511878367859305, pvalue=5.1234758101231843e-26)

In [30]:
pearsonr(dt['gs'],compute_metrics([tp1.apply_lesk_data('tokenized'), tp2.apply_lesk_data('tokenized')], ['jaccard']).do()[0])

PearsonRResult(statistic=-0.44025516061325287, pvalue=3.494129449862612e-23)

##### Testing lesk with lemmas

In [31]:
pearsonr(dt['gs'],compute_metrics([tp1.apply_lesk_lemmas_data('cleaned'), tp2.apply_lesk_lemmas_data('cleaned')], ['jaccard']).do()[0])

PearsonRResult(statistic=-0.46511878367859305, pvalue=5.1234758101231843e-26)

In [32]:
pearsonr(dt['gs'],compute_metrics([tp1.apply_lesk_lemmas_data('tokenized'), tp2.apply_lesk_lemmas_data('tokenized')], ['jaccard']).do()[0])

PearsonRResult(statistic=-0.43751386820493765, pvalue=6.946178670452206e-23)

#### Testing with name entities NLTK

In [33]:
pearsonr(dt['gs'],compute_metrics([tp1.name_entities_nltk('cleaned'), tp2.name_entities_nltk('cleaned')], ['jaccard']).do()[0])

PearsonRResult(statistic=-0.4679440754704449, pvalue=2.358310094026997e-26)

In [34]:
pearsonr(dt['gs'],compute_metrics([tp1.name_entities_nltk('tokenized'), tp2.name_entities_nltk('tokenized')], ['jaccard']).do()[0])

PearsonRResult(statistic=-0.4499943169285951, pvalue=2.8905921464511112e-24)

#### Name entities NLTK

In [35]:
pearsonr(dt['gs'],compute_metrics([tp1.name_entities_spacy(), tp2.name_entities_spacy()], ['jaccard']).do()[0])

PearsonRResult(statistic=-0.4493107049487957, pvalue=3.452252846323494e-24)

# Testing on training data

In [36]:
import pandas as pd
from text_processing import text_processing, compute_metrics
from scipy.stats import pearsonr
from collections import Counter
import io
europarl = pd.read_csv('Complementary Material/train/STS.input.SMTeuroparl.txt',sep='\t',header=None)
europarl['gs'] = pd.read_csv('Complementary Material/train/STS.gs.SMTeuroparl.txt',sep='\t',header=None)

vid = pd.read_csv('Complementary Material/train/STS.input.MSRvid.txt',sep='\t',header=None)
vid['gs'] = pd.read_csv('Complementary Material/train/STS.gs.MSRvid.txt',sep='\t',header=None)

with open('Complementary Material/train/STS.input.MSRpar.txt') as f:
    lines = f.readlines()
for index in range(len(lines)):
    lines[index] = lines[index].replace('\"', ' ')
par = pd.read_csv(io.StringIO(''.join(lines)), sep='\t',header=None, on_bad_lines='warn')
par['gs'] = pd.read_csv('Complementary Material/train/STS.gs.MSRpar.txt',sep='\t',header=None)

total = pd.concat([europarl, vid, par]).reset_index(drop=True)

In [37]:
eur_tp1 = text_processing(europarl[0])
eur_tp2 = text_processing(europarl[1])
pearsonr(europarl['gs'],compute_metrics([eur_tp1.mc_lemmatize_data('cleaned'), eur_tp2.mc_lemmatize_data('cleaned')], ['jaccard']).do()[0])


 -- Data hasn't been cleaned, to apply most_common_lemma to data is going to be cleaned with the default parameters --


 -- Data hasn't been cleaned, to apply most_common_lemma to data is going to be cleaned with the default parameters --



PearsonRResult(statistic=-0.44013235092459513, pvalue=3.963249990137324e-36)

In [38]:
vid_tp1 = text_processing(vid[0])
vid_tp2 = text_processing(vid[1])
pearsonr(vid['gs'],compute_metrics([vid_tp1.mc_lemmatize_data('cleaned'), vid_tp2.mc_lemmatize_data('cleaned')], ['jaccard']).do()[0])


 -- Data hasn't been cleaned, to apply most_common_lemma to data is going to be cleaned with the default parameters --


 -- Data hasn't been cleaned, to apply most_common_lemma to data is going to be cleaned with the default parameters --



PearsonRResult(statistic=-0.743585243920414, pvalue=6.8783591678791514e-133)

In [39]:
par_tp1 = text_processing(par[0])
par_tp2 = text_processing(par[1])
pearsonr(par['gs'],compute_metrics([par_tp1.mc_lemmatize_data('cleaned'), par_tp2.mc_lemmatize_data('cleaned')], ['jaccard']).do()[0])


 -- Data hasn't been cleaned, to apply most_common_lemma to data is going to be cleaned with the default parameters --


 -- Data hasn't been cleaned, to apply most_common_lemma to data is going to be cleaned with the default parameters --



PearsonRResult(statistic=-0.4607370310110359, pvalue=1.1031499896752255e-40)

In [40]:
total_tp1 = text_processing(total[0])
total_tp2 = text_processing(total[1])
pearsonr(total['gs'],compute_metrics([total_tp1.mc_lemmatize_data('cleaned'), total_tp2.mc_lemmatize_data('cleaned')], ['jaccard']).do()[0])


 -- Data hasn't been cleaned, to apply most_common_lemma to data is going to be cleaned with the default parameters --


 -- Data hasn't been cleaned, to apply most_common_lemma to data is going to be cleaned with the default parameters --



PearsonRResult(statistic=-0.605690059448885, pvalue=8.285888569412453e-224)

# Combining methods

In [41]:
total1 = text_processing(total[0])
total2 = text_processing(total[1])
t1,t2 = total1.clean_data(), total2.clean_data()
pearsonr(total['gs'],compute_metrics([t1, t2], ['jaccard']).do()[0])

PearsonRResult(statistic=-0.5710296807912365, pvalue=1.5887971093998278e-193)

In [42]:
t1,t2 = total_tp1.name_entities_nltk(data = t1), total_tp1.name_entities_nltk(data = t2)
pearsonr(total['gs'],compute_metrics([t1, t2], ['jaccard']).do()[0])

PearsonRResult(statistic=-0.5710296807912365, pvalue=1.5887971093998278e-193)

In [43]:
t1,t2 = total_tp1.lemmatize_data(data = t1), total_tp1.lemmatize_data(data = t2)
pearsonr(total['gs'],compute_metrics([t1, t2], ['jaccard']).do()[0])

PearsonRResult(statistic=-0.6028411708465875, pvalue=3.5388242418854675e-221)

In [44]:
t1,t2 = total_tp1.mc_lemmatize_data(data = t1), total_tp1.mc_lemmatize_data(data = t2)
pearsonr(total['gs'],compute_metrics([t1, t2], ['jaccard']).do()[0])

PearsonRResult(statistic=-0.6033214913285423, pvalue=1.2800280496346832e-221)

# Testing in all test_data

In [335]:
import pandas as pd
from text_processing import text_processing, compute_metrics
from scipy.stats import pearsonr
from collections import Counter
import io
europarl = pd.read_csv('Complementary Material/test-gold/STS.input.SMTeuroparl.txt',sep='\t',header=None)
europarl['gs'] = pd.read_csv('Complementary Material/test-gold/STS.gs.SMTeuroparl.txt',sep='\t',header=None)

vid = pd.read_csv('Complementary Material/test-gold/STS.input.MSRvid.txt',sep='\t',header=None)
vid['gs'] = pd.read_csv('Complementary Material/test-gold/STS.gs.MSRvid.txt',sep='\t',header=None)

with open('Complementary Material/test-gold/STS.input.MSRpar.txt', encoding='utf8') as f:
    lines = f.readlines()
for index in range(len(lines)):
    lines[index] = lines[index].replace('\"', ' ')
par = pd.read_csv(io.StringIO(''.join(lines)), sep='\t',header=None, on_bad_lines='warn')
par['gs'] = pd.read_csv('Complementary Material/test-gold/STS.gs.MSRpar.txt',sep='\t',header=None)

onwn = pd.read_csv('Complementary Material/test-gold/STS.input.surprise.OnWN.txt',sep='\t',header=None)
onwn['gs'] = pd.read_csv('Complementary Material/test-gold/STS.gs.surprise.OnWN.txt',sep='\t',header=None)

news = pd.read_csv('Complementary Material/test-gold/STS.input.surprise.SMTnews.txt',sep='\t',header=None)
news['gs'] = pd.read_csv('Complementary Material/test-gold/STS.gs.surprise.SMTnews.txt',sep='\t',header=None) 

total = pd.concat([europarl, vid, par, onwn, news]).reset_index(drop=True)

In [336]:
total1 = text_processing(total[0])
total2 = text_processing(total[1])
t1,t2 = total1.clean_data(), total2.clean_data()
pearsonr(total['gs'],compute_metrics([t1, t2], ['jaccard']).do()[0])

PearsonRResult(statistic=-0.5453285641443748, pvalue=2.3585890733699138e-240)

In [337]:
t1,t2 = total_tp1.mc_lemmatize_data(data = t1), total_tp1.mc_lemmatize_data(data = t2)
pearsonr(total['gs'],compute_metrics([t1, t2], ['jaccard']).do()[0])

PearsonRResult(statistic=-0.5768922741058112, pvalue=2.8794489176194942e-275)

# Experiments for extracting features

In [48]:
total[0][0], total[1][0]

('The leaders have now been given a new chance and let us hope they seize it.',
 "The leaders benefit aujourd' hui of a new luck and let's let them therefore seize it.")

In [209]:
r1,r2 = total1.mc_lemmatize_data(lemma = True)[0],total2.mc_lemmatize_data(lemma = True)[0]

In [205]:
r1

[Lemma('leader.n.01.leader'),
 Lemma('give.v.01.give'),
 Lemma('new.a.01.new'),
 Lemma('opportunity.n.01.chance'),
 Lemma('let.v.01.let'),
 None,
 Lemma('seize.v.01.seize')]

In [111]:
import itertools as it

In [188]:
combs = list(it.product(r1,r2))

In [213]:
r1

[Lemma('leader.n.01.leader'),
 Lemma('give.v.01.give'),
 Lemma('new.a.01.new'),
 Lemma('opportunity.n.01.chance'),
 Lemma('let.v.01.let'),
 None,
 Lemma('seize.v.01.seize')]

In [214]:
r2

[Lemma('leader.n.01.leader'),
 Lemma('profit.v.01.benefit'),
 None,
 None,
 Lemma('new.a.01.new'),
 Lemma('fortune.n.04.fortune'),
 Lemma('lashkar-e-taiba.n.01.Lashkar-e-Taiba'),
 Lemma('let.v.01.let'),
 Lemma('therefore.r.01.therefore'),
 Lemma('seize.v.01.seize')]

In [210]:
def path_similarities(lemmas1,lemmas2):
    combs = list(it.product(lemmas1,lemmas2))
    return [pair[0].synset().path_similarity(pair[1].synset()) for pair in combs if pair[0] and pair[1]]

In [237]:
Counter(path_similarities(r1,r2))

Counter({1.0: 4,
         0.1111111111111111: 7,
         0.125: 9,
         0.1: 6,
         0.08333333333333333: 2,
         0.25: 9,
         0.3333333333333333: 7,
         0.09090909090909091: 2,
         0.16666666666666666: 1,
         0.2: 1})

In [329]:
l1 = total1.lemmatize_data(r_pos_tag = False)[0]

In [330]:
l2 = total2.lemmatize_data(r_pos_tag = False)[0]

In [447]:
def get_synsets(lemma, tag):
        if not tag:
            synsets = wn.synsets(lemma)
        else:
            if lemma[1][0] in list(pos_map.keys()):
                synsets = wn.synsets(lemma[0], pos_map[lemma[1][0]])
            else:
                synsets = wn.synsets(lemma[0])
        return synsets

In [448]:
def path_similarity_word(lemma1,lemma2,tag = False):
    synsets1 = get_synsets(lemma1, tag)
    synsets2 = get_synsets(lemma2, tag)
    similarities = [syn1.path_similarity(syn2) for syn1 in synsets2 for syn2 in synsets2]
    if similarities != []:
        return max(similarities)
    return 0

In [441]:
l1,l2

(['leader', 'give', 'new', 'chance', 'let', 'hope', 'seize'],
 ['leader',
  'benefit',
  'aujourd',
  'hui',
  'new',
  'luck',
  'let',
  'let',
  'therefore',
  'seize'])

In [450]:
path_similarity_word(l1[0],l2[0])

1.0

In [456]:
def path_similarity_sentence(lemmas1, lemmas2, tag = False):
    if tag:
        lemmas1 = nltk.pos_tag(lemmas1)
        lemmas2 = nltk.pos_tag(lemmas2)
    mean1 = sum(max([path_similarity_word(l1,l2, tag) for l2 in lemmas2]) for l1 in lemmas1)/len(lemmas1)
    mean2 = sum(max([path_similarity_word(l2,l1, tag) for l1 in lemmas1]) for l2 in lemmas2)/len(lemmas2)
    if mean1 != 0 and mean2 != 0:
        return (2*mean1*mean2)/(mean1+mean2)
    

In [457]:
path_similarity_sentence(l1,l2,True)

1.0

In [465]:
def path_similarity(data = False, tag = False):
    # j_data = self.data if not data else data
    j_data = data
    result = []
    for n,row in enumerate(j_data):
        if n%50 == 0:
            print('Pairs analyzed: ', n)
        result.append(path_similarity_sentence(row[0],row[1], tag))
    return result

In [468]:
data[0]

array([list(['leader', 'give', 'new', 'chance', 'let', 'hope', 'seize']),
       list(['leader', 'benefit', 'aujourd', 'hui', 'new', 'luck', 'let', 'let', 'therefore', 'seize'])],
      dtype=object)

In [467]:
data = np.array([total1.lemmatize_data(), total2.lemmatize_data()], dtype=object).T
path_similarity(data)

Pairs analyzed:  0
Pairs analyzed:  50
Pairs analyzed:  100
Pairs analyzed:  150
Pairs analyzed:  200
Pairs analyzed:  250
Pairs analyzed:  300
Pairs analyzed:  350
Pairs analyzed:  400
Pairs analyzed:  450
Pairs analyzed:  500
Pairs analyzed:  550
Pairs analyzed:  600
Pairs analyzed:  650
Pairs analyzed:  700
Pairs analyzed:  750
Pairs analyzed:  800
Pairs analyzed:  850
Pairs analyzed:  900
Pairs analyzed:  950
Pairs analyzed:  1000
Pairs analyzed:  1050
Pairs analyzed:  1100
Pairs analyzed:  1150
Pairs analyzed:  1200
Pairs analyzed:  1250
Pairs analyzed:  1300
Pairs analyzed:  1350
Pairs analyzed:  1400
Pairs analyzed:  1450
Pairs analyzed:  1500
Pairs analyzed:  1550
Pairs analyzed:  1600


KeyboardInterrupt: 

In [279]:
similarities = {'wup':{},
                'path':{},
                'lch':{},
                'lin':{}}

maxi = {'wup':1, 'path':1, 'lin':1, 'lch':3}

v_pos = {'n','v'}

def get_synsets(lemma, tag):
        if not tag:
            synsets = wn.synsets(lemma)
        else:
            if lemma[1][0] in list(pos_map.keys()):
                synsets = wn.synsets(lemma[0], pos_map[lemma[1][0]])
            else:
                synsets = wn.synsets(lemma[0])
        return synsets
def similarity_word(method,lemma1,lemma2,tag = False):

    if lemma1 == lemma2:
        return maxi[method]

    if method in similarities:
        if (lemma1,lemma2) in similarities[method]:
            return similarities[method][(lemma1,lemma2)]

    synsets1 = get_synsets(lemma1, tag)
    synsets2 = get_synsets(lemma2, tag)
    
    if method == 'path':
        similarities_t = [syn1.path_similarity(syn2) for syn1 in synsets1 for syn2 in synsets2]
    elif method == 'wup':
        similarities_t = [syn1.wup_similarity(syn2) for syn1 in synsets1 for syn2 in synsets2]
    elif method == 'lch':
        similarities_t = [syn1.lch_similarity(syn2) for syn1 in synsets1 for syn2 in synsets2 if syn1.pos() == syn2.pos()]
    else:
        similarities_t = [syn1.lin_similarity(syn2, brown_ic) for syn1 in synsets1 for syn2 in synsets2 if syn1.pos() == syn2.pos() and syn1.pos() in v_pos]

    if similarities_t != []:
        similarities[method][(lemma1,lemma2)] = max(similarities_t)
        return similarities[method][(lemma1,lemma2)]
    return 0

def similarity_sentence(method,lemmas1, lemmas2, tag = False):
    if tag:
        lemmas1 = nltk.pos_tag(lemmas1)
        lemmas2 = nltk.pos_tag(lemmas2)
    mean1 = sum(max([similarity_word(method,l1,l2, tag) for l2 in lemmas2]) for l1 in lemmas1)/len(lemmas1)
    mean2 = sum(max([similarity_word(method,l2,l1, tag) for l1 in lemmas1]) for l2 in lemmas2)/len(lemmas2)
    if mean1 != 0 and mean2 != 0:
        return (2*mean1*mean2)/(mean1+mean2)
    return 0

def synset_similarity(method, data = False, tag = False):
    # j_data = self.data if not data else data
    j_data = data
    result = []
    for n,row in enumerate(j_data):
        if n%50 == 0:
            print('Pairs analyzed: ', n)
        result.append(similarity_sentence(method,row[0],row[1], tag))
    return result

In [280]:
nltk.download('wordnet_ic')
from nltk.corpus import wordnet_ic
brown_ic = wordnet_ic.ic('ic-brown.dat')

[nltk_data] Downloading package wordnet_ic to
[nltk_data]     C:\Users\mario\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!


In [312]:
data = np.array([total1.lemmatize_data(), total2.lemmatize_data()], dtype=object).T
res = synset_similarity('path', data[:10])

Pairs analyzed:  0


In [313]:
res

[0.7355679702048417,
 1.0,
 0.5572441742654507,
 0.9090909090909091,
 1.0,
 0.5482758620689654,
 0.6253369272237198,
 1.0,
 0.6802721088435374,
 0.9090909090909091]

In [182]:
len(data)

3108

In [315]:
compute_metrics([total1.lemmatize_data()[:10], total2.lemmatize_data()[:10]], ['synset_similarity'], {'method':'path'}).do()[0]

Pairs analyzed:  0


[0.7355679702048417,
 1.0,
 0.5572441742654507,
 0.9090909090909091,
 1.0,
 0.5482758620689654,
 0.6253369272237198,
 1.0,
 0.6802721088435374,
 0.9090909090909091]

In [317]:
t1,t2 = total1.lemmatized_data, total2.lemmatized_data

In [316]:
def nmalized_length_difference(sentece1, sentece2):
    return abs(len(sentece1)-len(sentece2)) / max(len(sentece1), len(sentece2))


In [319]:
length_difference(t1[0], t2[0])

0.3

In [330]:
compute_metrics([t1,t2], ['norm_lenght_diff']).do()[0]

[0.3,
 0.0,
 0.0,
 0.25,
 0.0,
 0.2857142857142857,
 0.0,
 0.0,
 0.14285714285714285,
 0.25,
 0.0,
 0.0,
 0.0,
 0.3,
 0.16666666666666666,
 0.0,
 0.2857142857142857,
 0.0,
 0.1111111111111111,
 0.0,
 0.16666666666666666,
 0.0,
 0.0,
 0.0,
 0.0,
 0.14285714285714285,
 0.14285714285714285,
 0.0,
 0.0,
 0.14285714285714285,
 0.0,
 0.0,
 0.3333333333333333,
 0.16666666666666666,
 0.0,
 0.0,
 0.0,
 0.125,
 0.0,
 0.0,
 0.16666666666666666,
 0.0,
 0.0,
 0.125,
 0.3333333333333333,
 0.25,
 0.0,
 0.3,
 0.0,
 0.0,
 0.0,
 0.1111111111111111,
 0.0,
 0.14285714285714285,
 0.125,
 0.0,
 0.0,
 0.2,
 0.125,
 0.0,
 0.0,
 0.0,
 0.3333333333333333,
 0.0,
 0.0,
 0.3,
 0.0,
 0.0,
 0.25,
 0.125,
 0.0,
 0.1111111111111111,
 0.0,
 0.2222222222222222,
 0.125,
 0.0,
 0.0,
 0.0,
 0.0,
 0.2,
 0.3333333333333333,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.2857142857142857,
 0.16666666666666666,
 0.0,
 0.0,
 0.2857142857142857,
 0.0,
 0.1111111111111111,
 0.2222222222222222,
 0.0,
 0.2857142857142857,
 0.33333333333333

In [151]:
import nltk

In [ ]:
def wup_similarity_word(lemma1,lemma2,tag = False):
    synsets1 = get_synsets(lemma1, tag)
    synsets2 = get_synsets(lemma2, tag)
    similarities = [syn1.path_similarity(syn2) for syn1 in synsets2 for syn2 in synsets2]
    if similarities != []:
        return max(similarities)
    return 0
def similarity_sentence(method,lemmas1, lemmas2, tag = False):
    if tag:
        lemmas1 = nltk.pos_tag(lemmas1)
        lemmas2 = nltk.pos_tag(lemmas2)
    
    mean1 = sum(max([path_similarity_word(l1,l2, tag) for l2 in lemmas2]) for l1 in lemmas1)/len(lemmas1)
    mean2 = sum(max([path_similarity_word(l2,l1, tag) for l1 in lemmas1]) for l2 in lemmas2)/len(lemmas2)
    if mean1 != 0 and mean2 != 0:
        return (2*mean1*mean2)/(mean1+mean2)
def similarity(method, data = False, tag = False):
    # j_data = self.data if not data else data
    j_data = data
    result = []
    for n,row in enumerate(j_data):
        if n%50 == 0:
            print('Pairs analyzed: ', n)
        result.append(similarity_sentence(method,row[0],row[1], tag))
    return result

In [414]:
total1.lemmatize_data(r_pos_tag = False)[0][1]

'give'

In [373]:
wn.synsets(total1.lemmatize_data(r_pos_tag = False)[0][1])

[Synset('give.n.01'),
 Synset('give.v.01'),
 Synset('yield.v.01'),
 Synset('give.v.03'),
 Synset('give.v.04'),
 Synset('give.v.05'),
 Synset('hold.v.03'),
 Synset('give.v.07'),
 Synset('give.v.08'),
 Synset('give.v.09'),
 Synset('give.v.10'),
 Synset('render.v.04'),
 Synset('impart.v.01'),
 Synset('establish.v.05'),
 Synset('give.v.14'),
 Synset('give.v.15'),
 Synset('sacrifice.v.01'),
 Synset('pass.v.05'),
 Synset('give.v.18'),
 Synset('give.v.19'),
 Synset('give.v.20'),
 Synset('give.v.21'),
 Synset('grant.v.05'),
 Synset('move_over.v.01'),
 Synset('feed.v.02'),
 Synset('contribute.v.02'),
 Synset('collapse.v.01'),
 Synset('give.v.27'),
 Synset('give.v.28'),
 Synset('give.v.29'),
 Synset('afford.v.04'),
 Synset('give.v.31'),
 Synset('give.v.32'),
 Synset('give.v.33'),
 Synset('give.v.34'),
 Synset('give.v.35'),
 Synset('give.v.36'),
 Synset('give.v.37'),
 Synset('give.v.38'),
 Synset('give.v.39'),
 Synset('give.v.40'),
 Synset('give.v.41'),
 Synset('give.v.42'),
 Synset('give.v.43'),

In [ ]:
def path

In [62]:
import nltk

In [65]:
t1,t2 = nltk.pos_tag(r1), nltk.pos_tag(r2)

In [53]:
from nltk.corpus import wordnet as wn

In [72]:
match = {'JJ':"a", 'JJ':"s", 'RB':"r", 'NN':"n", 'VB':"v"}

In [134]:
s1,s2 = wn.synsets(t1[2][0], match[t1[2][1]]), wn.synsets(t2[2][0], match[t2[2][1]])

In [135]:
s1,s2

([Synset('new.a.01'),
  Synset('fresh.s.04'),
  Synset('raw.s.12'),
  Synset('new.s.04'),
  Synset('new.s.05'),
  Synset('new.a.06'),
  Synset('newfangled.s.01'),
  Synset('new.s.08'),
  Synset('modern.s.05'),
  Synset('new.s.10'),
  Synset('new.s.11')],
 [Synset('new.a.01'),
  Synset('fresh.s.04'),
  Synset('raw.s.12'),
  Synset('new.s.04'),
  Synset('new.s.05'),
  Synset('new.a.06'),
  Synset('newfangled.s.01'),
  Synset('new.s.08'),
  Synset('modern.s.05'),
  Synset('new.s.10'),
  Synset('new.s.11')])

In [141]:
l = Counter([j for i in s1 for j in i.lemmas()]).most_common(1)[0][0]

In [147]:

for i in range(len(s1)):
    print(s1[i].lemmas())

[Lemma('new.a.01.new')]
[Lemma('fresh.s.04.fresh'), Lemma('fresh.s.04.new'), Lemma('fresh.s.04.novel')]
[Lemma('raw.s.12.raw'), Lemma('raw.s.12.new')]
[Lemma('new.s.04.new'), Lemma('new.s.04.unexampled')]
[Lemma('new.s.05.new')]
[Lemma('new.a.06.new')]
[Lemma('newfangled.s.01.newfangled'), Lemma('newfangled.s.01.new')]
[Lemma('new.s.08.New')]
[Lemma('modern.s.05.Modern'), Lemma('modern.s.05.New')]
[Lemma('new.s.10.new'), Lemma('new.s.10.young')]
[Lemma('new.s.11.new')]


In [157]:
s1[6].lemmas()[1]#.synset()

Lemma('newfangled.s.01.new')

In [98]:
def print_n(n,p):
    if p:
        return n
        


In [101]:
print(print_n(5,False))

None


In [244]:
w1,w2 = total1.clean_data()[0][0], total2.clean_data()[0][0]
w1,w2

('leaders', 'leaders')

In [265]:
tags1 =nltk.pos_tag(total1.clean_data()[0])
tags1

[('leaders', 'NNS'),
 ('given', 'VBN'),
 ('new', 'JJ'),
 ('chance', 'NN'),
 ('let', 'VBD'),
 ('hope', 'PRP'),
 ('seize', 'VB')]

In [270]:
tags1[0][0], tags1[0][1][0].lower()

('leaders', 'n')

In [271]:
synsets1 = wn.synsets(tags1[0][0], tags1[0][1][0].lower())
synsets1

[Synset('leadership.n.02'), Synset('leader.n.01'), Synset('drawing_card.n.02')]

In [260]:
[lemma for synset in synsets1 for lemma in synset.lemmas()]


[Lemma('drawing_card.n.02.leader'),
 Lemma('drawing_card.n.02.loss_leader'),
 Lemma('drawing_card.n.02.drawing_card'),
 Lemma('leader.n.01.leader'),
 Lemma('leadership.n.02.leaders'),
 Lemma('leadership.n.02.leadership')]

In [259]:
Counter([lemma for synset in synsets1 for lemma in synset.lemmas()]).most_common(1)[0][0].synset()

Synset('leader.n.01')

In [272]:
wnl = nltk.stem.WordNetLemmatizer()

In [276]:
wnl.lemmatize('leaders', 'n')

'leader'

In [283]:
g = text_processing(europarl[1]).lemmatize_data()


 -- Data hasn't been cleaned, to lemmatize the data is going to be cleaned with the default parameters --



In [171]:
from nltk.corpus.reader.wordnet import VERB, NOUN, ADJ, ADV

pos_map = {'N': NOUN,
                        'V':VERB,
                        'J':ADJ,
                        'R':ADV}

In [189]:
pos_map.values()

dict_values(['n', 'v', 'a', 'r'])

In [290]:
ps = nltk.pos_tag(g[0])

In [295]:
for i in ps:
    print(i[0],'----',i[1],'-->',wn.synsets(i[0], pos_map[i[1][0]]))

leader ---- NN --> [Synset('leader.n.01'), Synset('drawing_card.n.02')]
benefit ---- NN --> [Synset('benefit.n.01'), Synset('benefit.n.02'), Synset('benefit.n.03')]
aujourd ---- JJ --> []
hui ---- VBZ --> []
new ---- JJ --> [Synset('new.a.01'), Synset('fresh.s.04'), Synset('raw.s.12'), Synset('new.s.04'), Synset('new.s.05'), Synset('new.a.06'), Synset('newfangled.s.01'), Synset('new.s.08'), Synset('modern.s.05'), Synset('new.s.10'), Synset('new.s.11')]
luck ---- JJ --> []
let ---- NN --> [Synset('lashkar-e-taiba.n.01'), Synset('let.n.02')]
let ---- VB --> [Synset('let.v.01'), Synset('let.v.02'), Synset('permit.v.01'), Synset('get.v.03'), Synset('let.v.05'), Synset('lease.v.03')]
therefore ---- VB --> []
seize ---- VB --> [Synset('seize.v.01'), Synset('seize.v.02'), Synset('appropriate.v.02'), Synset('impound.v.01'), Synset('assume.v.06'), Synset('seize.v.06'), Synset('seize.v.07'), Synset('grab.v.06')]


# Feature Extraction

In [3]:
import pandas as pd
from text_processing import text_processing, compute_metrics
from scipy.stats import pearsonr
from collections import Counter
import io

from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np

# --------------- Training Data
europarl_Train = pd.read_csv('Complementary Material/train/STS.input.SMTeuroparl.txt',sep='\t',header=None)
europarl_Train['gs'] = pd.read_csv('Complementary Material/train/STS.gs.SMTeuroparl.txt',sep='\t',header=None)

vid_Train = pd.read_csv('Complementary Material/train/STS.input.MSRvid.txt',sep='\t',header=None)
vid_Train['gs'] = pd.read_csv('Complementary Material/train/STS.gs.MSRvid.txt',sep='\t',header=None)

with open('Complementary Material/train/STS.input.MSRpar.txt') as f:
    lines = f.readlines()
for index in range(len(lines)):
    lines[index] = lines[index].replace('\"', ' ')
par_Train = pd.read_csv(io.StringIO(''.join(lines)), sep='\t',header=None, on_bad_lines='warn')
par_Train['gs'] = pd.read_csv('Complementary Material/train/STS.gs.MSRpar.txt',sep='\t',header=None)

total_Train = pd.concat([europarl_Train, vid_Train, par_Train]).reset_index(drop=True)

# --------------- Testing Data
europarl_Test = pd.read_csv('Complementary Material/test-gold/STS.input.SMTeuroparl.txt',sep='\t',header=None)
europarl_Test['gs'] = pd.read_csv('Complementary Material/test-gold/STS.gs.SMTeuroparl.txt',sep='\t',header=None)

vid_Test = pd.read_csv('Complementary Material/test-gold/STS.input.MSRvid.txt',sep='\t',header=None)
vid_Test['gs'] = pd.read_csv('Complementary Material/test-gold/STS.gs.MSRvid.txt',sep='\t',header=None)

with open('Complementary Material/test-gold/STS.input.MSRpar.txt', encoding='utf8') as f:
    lines = f.readlines()
for index in range(len(lines)):
    lines[index] = lines[index].replace('\"', ' ')
par_Test = pd.read_csv(io.StringIO(''.join(lines)), sep='\t',header=None, on_bad_lines='warn')
par_Test['gs'] = pd.read_csv('Complementary Material/test-gold/STS.gs.MSRpar.txt',sep='\t',header=None)

onwn_Test = pd.read_csv('Complementary Material/test-gold/STS.input.surprise.OnWN.txt',sep='\t',header=None)
onwn_Test['gs'] = pd.read_csv('Complementary Material/test-gold/STS.gs.surprise.OnWN.txt',sep='\t',header=None)

news_Test = pd.read_csv('Complementary Material/test-gold/STS.input.surprise.SMTnews.txt',sep='\t',header=None)
news_Test['gs'] = pd.read_csv('Complementary Material/test-gold/STS.gs.surprise.SMTnews.txt',sep='\t',header=None) 

total_Test = pd.concat([europarl_Test, vid_Test, par_Test, onwn_Test, news_Test]).reset_index(drop=True)

In [117]:
import string
import nltk
lowercase = True
stopwords=set(nltk.corpus.stopwords.words('english'))
signs = string.punctuation
minwords_len = 2

In [169]:
Train1 = text_processing(total_Train[0])
Train2 = text_processing(total_Train[1])
Train1.clean_data(auto=False, lowercase = lowercase, signs = signs),Train2.clean_data(auto=False, lowercase = lowercase, signs = signs)
#Train1.clean_data(),Train2.clean_data()
_,_ = Train1.lemmatize_data(), Train2.lemmatize_data() 

In [174]:
save_path = 'Complementary Material/Extracted Features/Train/unigram_semi_clean'

In [175]:
X = compute_metrics([Train1.cleaned_data,Train2.cleaned_data],['unigram']).do(save_path)

In [355]:
X = np.moveaxis(np.array(X),0,-1)

In [357]:
X.shape

(2234, 1)

In [65]:
X = np.load('Complementary Material/Extracted Features/Train/PathSym_lemmas.npy')

In [67]:
X = np.moveaxis(np.array(X),0,-1)
X.shape

(2234, 1)

In [113]:
Test1 = text_processing(total_Test[0])
Test2 = text_processing(total_Test[1])
Test1.clean_data(),Test2.clean_data()
_,_ = Test1.lemmatize_data(), Test2.lemmatize_data() 

In [167]:
save_path = 'Complementary Material/Extracted Features/Test/trigram_comp_clean'

In [168]:
X_test = compute_metrics([Test1.cleaned_data,Test2.cleaned_data],['trigram']).do(save_path)

In [364]:
X_test = np.moveaxis(np.array(X_test),0,-1)

In [90]:
X_test = np.load('Complementary Material/Extracted Features/Test/PathSym_lemmas.npy')
X_test = np.moveaxis(np.array(X_test),0,-1)
X_test.shape

(3108, 1)

In [190]:
feature = 'trigram'

Train1 = text_processing(total_Train[0])
Train2 = text_processing(total_Train[1])
#Train1.clean_data(auto=False, lowercase = lowercase, signs = signs),Train2.clean_data(auto=False, lowercase = lowercase, signs = signs)
Train1.clean_data(),Train2.clean_data()
_,_ = Train1.lemmatize_data(), Train2.lemmatize_data() 

save_train_path = 'Complementary Material/Extracted Features/Train/'+feature+'_comp_clean'
X = compute_metrics([Train1.cleaned_data,Train2.cleaned_data],[feature]).do(save_train_path)

Test1 = text_processing(total_Test[0])
Test2 = text_processing(total_Test[1])
#Test1.clean_data(auto=False, lowercase = lowercase, signs = signs),Test2.clean_data(auto=False, lowercase = lowercase, signs = signs)
Test1.clean_data(),Test2.clean_data()
_,_ = Test1.lemmatize_data(), Test2.lemmatize_data() 

save_test_path = 'Complementary Material/Extracted Features/Test/'+feature+'_comp_clean'
X_test = compute_metrics([Test1.cleaned_data,Test2.cleaned_data],[feature]).do(save_test_path)

# Classifiers Test

In [251]:
features_to_try = ['LinSym_lemmas', 'LchSym_lemmas', 'PathSym_lemmas', 'jaccard_lemmas', 'cosine_lemmas', 'unigram_lemmas', 'bigram_lemmas', 'trigram_lemmas', 'jaccard_semi_clean', 'bigram_semi_clean', 'trigram_semi_clean', 'jaccard_comp_clean', 'unigram_comp_clean', 'bigram_comp_clean']
train_feats = np.load('Complementary Material/Extracted Features/Train/'+features_to_try[0]+'.npy')
for n in range(1, len(features_to_try)):
    train_feats = np.concatenate((train_feats, np.load('Complementary Material/Extracted Features/Train/'+features_to_try[n]+'.npy')))
train_feats = np.moveaxis(np.array(train_feats),0,-1)

y = total_Train['gs'].values

regr = make_pipeline(StandardScaler(), SVR(kernel = 'rbf', C=1, epsilon=0, gamma = 'scale', tol=1e-1))
regr.fit(train_feats, y)

test_feats = np.load('Complementary Material/Extracted Features/Test/'+features_to_try[0]+'.npy')
for n in range(1, len(features_to_try)):
    test_feats = np.concatenate((test_feats, np.load('Complementary Material/Extracted Features/Test/'+features_to_try[n]+'.npy')))
test_feats = np.moveaxis(np.array(test_feats),0,-1)

pearsonr(total_Test['gs'].values,regr.predict(test_feats))

PearsonRResult(statistic=0.7406540348318249, pvalue=0.0)

In [177]:
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingRegressor

In [252]:
features_to_try = ['LinSym_lemmas', 'LchSym_lemmas', 'PathSym_lemmas', 'jaccard_lemmas', 'cosine_lemmas', 'unigram_lemmas', 'bigram_lemmas', 'trigram_lemmas', 'jaccard_semi_clean', 'bigram_semi_clean', 'trigram_semi_clean', 'jaccard_comp_clean', 'unigram_comp_clean', 'bigram_comp_clean']
#features_to_try = ['LinSym_lemmas', 'LchSym_lemmas', 'PathSym_lemmas', 'jaccard_lemmas', 'cosine_lemmas', 'unigram_lemmas', 'bigram_lemmas', 'trigram_lemmas', 'jaccard_semi_clean', 'jaccard_comp_clean', 'unigram_comp_clean']
train_feats = np.load('Complementary Material/Extracted Features/Train/'+features_to_try[0]+'.npy')
for n in range(1, len(features_to_try)):
    train_feats = np.concatenate((train_feats, np.load('Complementary Material/Extracted Features/Train/'+features_to_try[n]+'.npy')))
train_feats = np.moveaxis(np.array(train_feats),0,-1)

y = total_Train['gs'].values

gbr = GradientBoostingRegressor(random_state=17)
gbr.fit(train_feats, y)


test_feats = np.load('Complementary Material/Extracted Features/Test/'+features_to_try[0]+'.npy')
for n in range(1, len(features_to_try)):
    test_feats = np.concatenate((test_feats, np.load('Complementary Material/Extracted Features/Test/'+features_to_try[n]+'.npy')))
test_feats = np.moveaxis(np.array(test_feats),0,-1)

pearsonr(total_Test['gs'].values,gbr.predict(test_feats))

PearsonRResult(statistic=0.7383876997011208, pvalue=0.0)

In [179]:
from sklearn.neural_network import MLPRegressor

In [331]:
features_to_try = ['LinSym_lemmas', 'LchSym_lemmas', 'PathSym_lemmas', 'jaccard_lemmas', 'cosine_lemmas', 'unigram_lemmas', 'bigram_lemmas', 'trigram_lemmas', 'jaccard_semi_clean', 'bigram_semi_clean', 'trigram_semi_clean', 'jaccard_comp_clean', 'unigram_comp_clean', 'bigram_comp_clean']
#features_to_try = ['LinSym_lemmas', 'LchSym_lemmas', 'PathSym_lemmas', 'jaccard_lemmas', 'cosine_lemmas', 'unigram_lemmas', 'bigram_lemmas', 'trigram_lemmas', 'jaccard_semi_clean', 'bigram_semi_clean', 'trigram_semi_clean']
train_feats = np.load('Complementary Material/Extracted Features/Train/'+features_to_try[0]+'.npy', allow_pickle=True)
for n in range(1, len(features_to_try)):
    train_feats = np.concatenate((train_feats, np.load('Complementary Material/Extracted Features/Train/'+features_to_try[n]+'.npy')))
train_feats = np.moveaxis(np.array(train_feats),0,-1)

y = total_Train['gs'].values

#mlp = MLPRegressor(hidden_layer_sizes=(100,500,1000,500,100), random_state=5, max_iter=900)
mlp = MLPRegressor(hidden_layer_sizes=(100,100,100, 100), random_state=3, max_iter=200)
mlp.fit(train_feats, y)

test_feats = np.load('Complementary Material/Extracted Features/Test/'+features_to_try[0]+'.npy', allow_pickle=True)
for n in range(1, len(features_to_try)):
    test_feats = np.concatenate((test_feats, np.load('Complementary Material/Extracted Features/Test/'+features_to_try[n]+'.npy')))
test_feats = np.moveaxis(np.array(test_feats),0,-1)

pearsonr(total_Test['gs'].values,mlp.predict(test_feats))

PearsonRResult(statistic=0.7518254148216057, pvalue=0.0)